# Контекстные менеджеры и генераторы

## Контекстные менеджеры

На занятии про исключения мы впервые столкнулись с контекстными менеджерами. Напомним, что контекстные менеджеры - это объекты, которые могут быть использованы в конструкции `with`. Обычно контекстные менеджеры используются для гарантированного выполнения каких-либо действий после выполнения вашего блока кода. К типичным действиям, выполняемым с помощью контекстных менеджеров, относится освобождение ресурсов ОС: закрытие файлового дескриптора, закрытие сетевого соединения и т.д.

Ярким примером использования контекстного менеджера является работа с файлами:

In [ ]:
path_to_file = "./important_info.txt"

with open(path_to_file, encoding="utf-8") as file:
    print("".join(file.readlines()))

В данном случае использование `with` с результатом выполнения функции `open` гарантирует закрытие файлового дескриптора в независимости от того, произойдет ли какая-либо ошибка в теле блока `with`.

До сегодняшнего дня работа контекстных менеджеров была похожа на магию. Однако на самом деле никакой магии нет. Все дело в протоколе контекстного менеджера. Любой объект, который удовлетворяет этому протоколу, может использоваться в заголовке блока `with`. Рассмотрим протокол контекстного менеджера подробнее:

In [ ]:
from types import TracebackType
from typing import Optional


class MyContextManager:
    def __enter__(self) -> None:
        print("call __enter__")

    def __exit__(
        self,
        exc_type: Optional[type[BaseException]],
        exc_val: Optional[BaseException],
        exc_tb: Optional[TracebackType],
    ) -> bool:
        print(
            "call __exit__",
            f"exc_type: {exc_type}",
            f"exc_val: {exc_val}",
            f"exc_tb: {exc_tb}",
            sep="\n",
        )
        return False

Для того, чтобы объект удовлетворял протоколу контекстного менеджера, для него необходимо реализовать специальные методы `__enter__` и `__exit__`:
- `__enter__` - вызывается до выполнения тела блока `with`. В этом методе обычно происходит подготовка контекстного менеджера. Сам метод не принимает на вход никаких параметров, кроме данного экземпляра класса. В качестве возвращаемого значения метода `__enter__` можно использовать любой объект. Однако на практике чаще всего в качестве результата выполнения `__enter__` используется `None`, или данный экземпляр класса.
- `__exit__` - вызывается после выполнения тела блока `with` или после возникновения исключения в нем. Помимо данного экземпляра класса, в качестве параметров в этот метод неявно могут быть переданы тип возбужденного исключения, само возбужденное исключение и трейсбек. По умолчанию в качестве этих аргументов передается `None`. `__exit__` возвращает `True` в том случае, если возбужденное исключение было обработано в данном методе, и `False` - иначе.

In [ ]:
with MyContextManager():
    print("do something")

In [ ]:
with MyContextManager():
    raise Exception("something was broken")

В некоторых случаях контекстные менеджеры используется не только для захвата ресурсов, но и для предоставления интерфейса по работе с этими ресурсами. В таких случаях в `__enter__` полезно возвращать сам созданный экземпляр. Тогда пользователь сможет связать результат выполнения `__enter__` с переменной и использовать ее в теле блока `with`:

In [ ]:
from typing import Any, Self


class ConnectionDB:
    _is_closed: bool

    def __init__(self) -> None:
        self._is_closed = True

    def __enter__(self) -> Self:
        return self.connect()

    def __exit__(self, *_: Any) -> bool:
        self.close()
        return False
    
    def connect(self) -> Self:
        if not self._is_closed:
            raise RuntimeError("connection is already opened")
        
        print("open connection to database")
        self._is_closed = False
        return self

    def close(self) -> None:
        if self._is_closed:
            raise RuntimeError("connection is already closed")

        print("close connection to database")
        self._is_closed = True

    def get_user_amount(self) -> int:
        if self._is_closed:
            raise RuntimeError(
                "impossible to send request without opened connection"
            )

        return 42

In [ ]:
with ConnectionDB() as connection:
    user_amount = connection.get_user_amount()
    print(f"user_amount: {user_amount}")

Альтернативное решение:

In [ ]:
from typing import Any


class ConnectionDB:
    _is_closed: bool

    def __init__(self) -> None:
        self._is_closed = True

    def connect(self) -> None:
        if not self._is_closed:
            raise RuntimeError("connection is already opened")
        
        print("open connection to database")
        self._is_closed = False

    def close(self) -> None:
        if self._is_closed:
            raise RuntimeError("connection is already closed")

        print("close connection to database")
        self._is_closed = True

    def get_user_amount(self) -> int:
        if self._is_closed:
            raise RuntimeError(
                "impossible to send request without opened connection"
            )

        return 42


class ConnectionDBManager:
    _connection: Optional[ConnectionDB]

    def __enter__(self) -> ConnectionDB:
        self._connection = ConnectionDB()
        self._connection.connect()
        return self._connection

    def __exit__(self, *_: Any) -> bool:
        self._connection.close()
        self._connection = None
        return False

In [ ]:
with ConnectionDBManager() as connection:
    user_amount = connection.get_user_amount()
    print(f"user_amount: {user_amount}")

## Генераторы

На лекции мы рассмотрели протокол итерируемого объекта. До этого мы уже встречались с итерируемыми объектами, и все они были коллекциями, например, списки, словари и т.д. У всех них есть особенность: они хранят данные в памяти. Но в некоторых случаях эффективнее не хранить все данные, а вычислять их на лету, особенно, когда объемы данных слишком большие. 

Примером объекта, который не хранит в себе данные, а вычисляет их на лету, является встроенный объект `map`. Этот объект позволяет применить некоторую функцию ко всем элементам некоторого итерируемого объекта на лету, не создавая новую коллекцию.

In [ ]:
import sys

objects_amount = 100
squares = [i ** 2 for i in range(objects_amount)]
squares_map = map(lambda x: x ** 2, range(objects_amount))

print(
    f"size of list: {sys.getsizeof(squares)} bytes",
    f"size of map: {sys.getsizeof(squares_map)} bytes",
    sep="\n",
)

In [ ]:
for square in squares_map:
    print(square, end=" ")

for square in squares_map:
    print(square, end=" ")

Как видно из данного примера, объект `map` ведет себя как итератор. Фактически `map` и является итератором. Однако каждый раз вручную создавать итератор для того, чтобы вычислять некоторые значения на лету - утомительное занятие. Поэтому в Python существуют специальные объекты - генераторы. Фактически, генераторы являются частными случаями итераторов. Они также реализуют специальные методы `__iter__` и `__next__`. Однако они это делают неявно, и вам не придется определять их вручную. Также генераторы обладают дополнительными методами, но их в нашем курсе мы рассматривать не будем.

Простейший способ создать генератор - использовать генераторное выражение. Синтаксис генераторного выражения очень похож на уже знакомые нам списковые, словарные и множественные включения, однако вместо квадратных и фигурных скобочек используются круглые скобки:

In [ ]:
import sys

objects_amount = 1000
squares = [i ** 2 for i in range(objects_amount)]
squares_gen = (i ** 2 for i in range(objects_amount))

print(
    f"size of list: {sys.getsizeof(squares)} bytes",
    f"size of gen: {sys.getsizeof(squares_gen)} bytes",
    sep="\n",
)

In [ ]:
print(type(squares_gen).__name__)

In [ ]:
for square in squares_gen:
    print(square, end=" ")

for square in squares_gen:
    print(square, end=" ")

Генераторные выражения полезны, но они имеют существенное ограничение: мы не можем реализовать произвольную по сложности логику создания и обработки данных. Для обхода этого ограничения в Python существуют генераторные функции. Чтобы сделать обычную функцию генераторной, необходимо использовать ключевое слово `yield` в теле этой функции:

In [ ]:
from typing import Generator


def get_fibonachi_sequence() -> Generator[int, None, None]:
    num1, num2 = 0, 1

    while True:
        yield num2
        num1, num2 = num2, num1 + num2

In [ ]:
fibonachi_gen = get_fibonachi_sequence()

print(
    f"func type: {type(get_fibonachi_sequence).__name__}",
    f"func res type: {type(fibonachi_gen).__name__}",
    sep="\n",
)

for _ in range(5):
    print(next(fibonachi_gen))

In [ ]:
next(fibonachi_gen)

С помощью функции `get_fibonachi_sequence()` мы получили бесконечный генератор чисел из последовательности Фибоначчи.

Часто в теле генераторных функциях приходится итерироваться по некоторым итерируемым объектам и "производить" данные из этих итерируемых объектов. Специально для таких случаев существует конструкция `yield from`:

In [ ]:
from typing import Generator


def pyramid_range(stop: int) -> Generator[int, None, None]:
    yield from range(stop)
    yield from range(stop, -1, -1)

In [ ]:
for i in pyramid_range(3):
    print(i, end=" ")

Обращаем ваше внимание, что генераторы - это частные случаи итераторов. Т.е. генераторы одноразовые: вы сможете сгенерировать значения с помощью данного генератора всего один раз. При повторном использовании генератора с функцией `next()` вы будете получать исключение `StopIteration`:

In [ ]:
pyramid_gen = pyramid_range(3)

for i in pyramid_gen:
    print(i, end=" ")
    
    if i == 3:
        break
    
print()

for i in pyramid_gen:
    print(i, end=" ")

In [ ]:
pyramid_gen = pyramid_range(3)

for i in pyramid_gen:
    print(i, end=" ")

next(pyramid_gen)

## Контекстные менеджеры и генераторы

Думаем, вы оценили всю избыточность синтаксиса при определении своих контекстных менеджеров. Когда контекстные менеджеры делают очень простые вещи, определять целый класс со специальными методами для этих целей не нужно. Вместо этого можно описать контекстный менеджер с помощью генератора и декоратора `contextmanager` из модуля стандартной библиотеки `contextlib`.

Давайте перепишем наш пример с `ConnectionDB`, используя новый подход:

In [ ]:
from contextlib import contextmanager
from typing import Generator


class ConnectionDB:
    _is_closed: bool

    def __init__(self) -> None:
        self._is_closed = True

    def connect(self) -> None:
        if not self._is_closed:
            raise RuntimeError("connection is already opened")
        
        print("open connection to database")
        self._is_closed = False

    def close(self) -> None:
        if self._is_closed:
            raise RuntimeError("connection is already closed")

        print("close connection to database")
        self._is_closed = True

    def get_user_amount(self) -> int:
        if self._is_closed:
            raise RuntimeError(
                "impossible to send request without opened connection"
            )

        return 42


@contextmanager
def create_connection() -> Generator[ConnectionDB, None, None]:
    connection = ConnectionDB()
    connection.connect()

    try:
        yield connection

    finally:
        connection.close()

In [ ]:
with create_connection() as connection:
    user_amount = connection.get_user_amount()
    print(f"user_amount: {user_amount}")

Суть данного подхода заключается в следующем. Весь код, который расположен до `yield`, соответствует телу метода `__enter__`. Инструкция `yield` соответствует инструкции `return` в теле `__enter__`. Весь код, который расположен после инструкции `yield`, соответствует коду в теле метода `__exit__`. `try`-`finally` добавлен для гарантии выполнения кода, соответствующего освобождению ресурсов.

Как видим из примера, при гораздо меньших усилиях нам удалось достичь того же поведения, что и в исходном примере.